### Vehicle Detection Project -- Classification Part

Zikri Bayraktar, Ph.D.

This project has two parts ( in this notebook, we will focus on Part 1 ):
1. Train a classifier to identify cars on an image of a road.
2. Create a pipeline to detect vehicles in a video stream.

#### Part 1: Train a classifier to identify cars vs. non-cars (i.e. background)

In [1]:
import numpy as np
import cv2
import glob
import os
import time

from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from skimage.feature import hog

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [2]:
# Read in all available car images for training classifier:

vehicle_list=[]
for foldername in os.listdir('folder_vehicles'):
    for filename in glob.glob('folder_vehicles/'+str(foldername)+'/*.png'):
        vehicle_list.append(filename)

In [3]:
# Read in all available nonCar images for training classifier:

non_vehicle_list=[]
for filename in glob.glob('folder_non_vehicles/*.png'):
    non_vehicle_list.append(filename)

#### Some useful functions from lectures

In [4]:
# Define a function to compute binned color features.
# This function simply resizes image to a smaller size and creates a 1D feature vector to return.
# By shrinking the image, we are reducing the dimensions while still maintaining some relavant 
# features to help us identify cars visually.
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    #print('bin_spatial ' + str(max(features)))   --> range of [0,1]
    return features

In [5]:
# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the RGB channels separately
    c1hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    c2hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    c3hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Generating bin centers
    bin_edges = c1hist[1]
    bin_centers = (bin_edges[1:]  + bin_edges[0:len(bin_edges)-1])/2
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((c1hist[0], c2hist[0], c3hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    #return c1hist, c2hist, c3hist, bin_centers, hist_features
    #print('hist ' + str(hist_features))
    return hist_features

In [12]:
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_color_features(imgs, cspace='RGB', spatial_size=(32, 32), hist_bins=32, hist_range=(0, 256)):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        
        #image = image*255  
        
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      
        
        #print(feature_image)
        
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #print('spatial ' + str(max(spatial_features)))
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        #print('hist ' + str(hist_features))
        # Append the new feature vector to the features list
        features.append(np.concatenate((spatial_features, hist_features)))
    # Return list of feature vectors
    return features

In [13]:
# Define a function to extract HOG features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_hog_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
#        image=image*255
        
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      

        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        # Append the new feature vector to the features list
        features.append(hog_features)
    # Return list of feature vectors
    return features


In [14]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    draw_img = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(draw_img, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return draw_img

In [15]:
# Define a function to search for template matches
# and return a list of bounding boxes
def find_matches(img, template_list):
    # Define an empty list to take bbox coords
    bbox_list = []
    # Define matching method
    # Other options include: cv2.TM_CCORR_NORMED', 'cv2.TM_CCOEFF', 'cv2.TM_CCORR',
    #         'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED'
    method = cv2.TM_CCOEFF_NORMED
    # Iterate through template list
    for temp in template_list:
        # Read in templates one by one
        tmp = mpimg.imread(temp)
        # Use cv2.matchTemplate() to search the image
        result = cv2.matchTemplate(img, tmp, method)
        # Use cv2.minMaxLoc() to extract the location of the best match
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        # Determine a bounding box for the match
        w, h = (tmp.shape[1], tmp.shape[0])
        if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
            top_left = min_loc
        else:
            top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        # Append bbox position to list
        bbox_list.append((top_left, bottom_right))
        # Return the list of bounding boxes
        
    return bbox_list

In [16]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',
                       transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

### Extract COLOR features:

In [17]:
# Extract Color Features:
spatial = 32
histbin = 32
colorspace = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb


car_color_features = extract_color_features(vehicle_list, cspace=colorspace, spatial_size=(spatial, spatial),
                        hist_bins=histbin, hist_range=(0, 1.0))


notcar_color_features = extract_color_features(non_vehicle_list, cspace=colorspace, spatial_size=(spatial, spatial),
                        hist_bins=histbin, hist_range=(0, 1.0))

[[[ 0.43414512  0.49243101  0.48958921]
  [ 0.42860395  0.49078971  0.48829091]
  [ 0.41756469  0.49306855  0.48567   ]
  ..., 
  [ 0.50978434  0.5195865   0.41375223]
  [ 0.45969805  0.51894903  0.41767147]
  [ 0.43337649  0.51534766  0.41703451]]

 [[ 0.68083924  0.47871065  0.49421844]
  [ 0.68476075  0.47871071  0.49421847]
  [ 0.6612314   0.47871065  0.49421844]
  ..., 
  [ 0.47540787  0.51334006  0.40659937]
  [ 0.47125098  0.51630396  0.41115564]
  [ 0.45370984  0.51762646  0.41441354]]

 [[ 0.63905883  0.46376285  0.51778257]
  [ 0.65082353  0.46376285  0.51778257]
  [ 0.61989808  0.46344405  0.51974219]
  ..., 
  [ 0.4791373   0.52186531  0.41113126]
  [ 0.53840786  0.5215466   0.41309091]
  [ 0.54949021  0.52482921  0.41568753]]

 ..., 
 [[ 0.2376353   0.47875819  0.50531489]
  [ 0.21964706  0.48039949  0.50661319]
  [ 0.20513335  0.48235953  0.50595188]
  ..., 
  [ 0.26451373  0.48755464  0.49679074]
  [ 0.2609961   0.48167449  0.49877468]
  [ 0.2692863   0.48135573  0.50073

[[[ 0.19522354  0.48103699  0.50269395]
  [ 0.18439609  0.4859609   0.50658882]
  [ 0.1692      0.49120355  0.50852412]
  ..., 
  [ 0.11033334  0.48564214  0.5085485 ]
  [ 0.11425491  0.48564214  0.5085485 ]
  [ 0.11817648  0.48564214  0.50854844]]

 [[ 0.19174902  0.48071826  0.50465357]
  [ 0.18439609  0.4859609   0.50658882]
  [ 0.1692      0.49120355  0.50852412]
  ..., 
  [ 0.11425491  0.48564214  0.5085485 ]
  [ 0.11425491  0.48564214  0.5085485 ]
  [ 0.11817648  0.48564214  0.50854844]]

 [[ 0.18390588  0.48071826  0.50465357]
  [ 0.18047452  0.4859609   0.50658882]
  [ 0.17356864  0.49088478  0.51048374]
  ..., 
  [ 0.12371765  0.48728344  0.50984681]
  [ 0.11979609  0.48728344  0.50984675]
  [ 0.11817648  0.48564214  0.50854844]]

 ..., 
 [[ 0.36250982  0.48199326  0.49681506]
  [ 0.36044317  0.48067069  0.49355713]
  [ 0.36067843  0.47770688  0.48900089]
  ..., 
  [ 0.41023532  0.49270222  0.47653317]
  [ 0.41298431  0.49074218  0.47719449]
  [ 0.4152863   0.48910087  0.47589

[[[ 0.5291059   0.50860631  0.44709015]
  [ 0.53974122  0.51220769  0.44772714]
  [ 0.54600787  0.51612777  0.44640452]
  ..., 
  [ 0.21820784  0.50938642  0.47646019]
  [ 0.20134902  0.50742638  0.4771215 ]
  [ 0.20640001  0.50382501  0.47648451]]

 [[ 0.52914906  0.51416773  0.44706583]
  [ 0.54483533  0.51416773  0.44706583]
  [ 0.55777264  0.51612771  0.44640446]
  ..., 
  [ 0.21635294  0.51070899  0.47971812]
  [ 0.21036471  0.50938642  0.47646019]
  [ 0.21424314  0.50382501  0.47648451]]

 [[ 0.54050982  0.5200479   0.44508189]
  [ 0.55272156  0.51972914  0.44704151]
  [ 0.56448627  0.51972914  0.44704151]
  ..., 
  [ 0.22698824  0.5143103   0.48035511]
  [ 0.22212942  0.50938642  0.47646019]
  [ 0.21861178  0.5035063   0.47844413]]

 ..., 
 [[ 0.15253334  0.48071826  0.50465357]
  [ 0.15758432  0.47711688  0.50401658]
  [ 0.16380785  0.47547558  0.50271827]
  ..., 
  [ 0.13004707  0.46599409  0.50406522]
  [ 0.13166668  0.46763539  0.50536352]
  [ 0.13260393  0.47255927  0.50925

[[[ 0.13085884  0.47939569  0.50139564]
  [ 0.12855688  0.48103699  0.50269395]
  [ 0.12350589  0.48463833  0.50333095]
  ..., 
  [ 0.17003138  0.47383431  0.50141996]
  [ 0.18273334  0.47875819  0.50531489]
  [ 0.18827452  0.48039949  0.50661319]]

 [[ 0.13365099  0.48299703  0.50203264]
  [ 0.13365099  0.48299703  0.50203264]
  [ 0.1286      0.4865984   0.50266963]
  ..., 
  [ 0.17395294  0.47383431  0.50141996]
  [ 0.1850353   0.47711688  0.50401658]
  [ 0.18827452  0.48039949  0.50661319]]

 [[ 0.14432941  0.49215981  0.50264525]
  [ 0.14432941  0.49215981  0.50264525]
  [ 0.1381059   0.49380109  0.50394356]
  ..., 
  [ 0.1845451   0.47187424  0.50208127]
  [ 0.18891373  0.4715555   0.5040409 ]
  [ 0.18891373  0.4715555   0.5040409 ]]

 ..., 
 [[ 0.19172943  0.4695479   0.49360579]
  [ 0.19407451  0.47346801  0.49228317]
  [ 0.20034119  0.47738811  0.49096051]
  ..., 
  [ 0.51857257  0.50213623  0.46408978]
  [ 0.52362359  0.49853483  0.46345276]
  [ 0.52362359  0.49853483  0.46345

[[[ 0.5138157   0.49434355  0.47783148]
  [ 0.35650587  0.49466231  0.47587186]
  [ 0.33022746  0.49662232  0.47521055]
  ..., 
  [ 0.43977258  0.50519508  0.49083886]
  [ 0.39893726  0.50355381  0.48954058]
  [ 0.36040393  0.5002712   0.48694396]]

 [[ 0.48636472  0.49434352  0.47783148]
  [ 0.37336472  0.49662232  0.47521055]
  [ 0.36434904  0.49466228  0.47587186]
  ..., 
  [ 0.38879216  0.50519514  0.49083889]
  [ 0.36526275  0.50519514  0.49083889]
  [ 0.35256082  0.5002712   0.48694393]]

 [[ 0.44440001  0.49630359  0.47717017]
  [ 0.39022353  0.49858239  0.47454923]
  [ 0.4008157   0.49662232  0.47521055]
  ..., 
  [ 0.33849412  0.50191253  0.48824227]
  [ 0.3357451   0.50387257  0.48758096]
  [ 0.34657255  0.49894869  0.48368603]]

 ..., 
 [[ 0.12416471  0.48696467  0.51180643]
  [ 0.12020001  0.48140329  0.51183075]
  [ 0.12015687  0.47584188  0.51185507]
  ..., 
  [ 0.08060392  0.48167449  0.49877468]
  [ 0.15999609  0.48098946  0.4915975 ]
  [ 0.26219216  0.47802562  0.48704

[[[ 0.83700788  0.52114671  0.47249228]
  [ 0.84759998  0.51918674  0.47315362]
  [ 0.85819215  0.51722664  0.47381493]
  ..., 
  [ 0.16321962  0.49826363  0.47650886]
  [ 0.1680353   0.49762613  0.4804281 ]
  [ 0.18024705  0.49730739  0.48238772]]

 [[ 0.79593337  0.49730736  0.48238772]
  [ 0.80147451  0.49894869  0.48368603]
  [ 0.80774122  0.50286877  0.48236337]
  ..., 
  [ 0.3333137   0.4888297   0.48895225]
  [ 0.28832158  0.49015221  0.49221015]
  [ 0.26871374  0.49015221  0.49221015]]

 [[ 0.70165884  0.46107021  0.50017029]
  [ 0.701702    0.46663159  0.50014597]
  [ 0.70178825  0.47775441  0.50009733]
  ..., 
  [ 0.58160394  0.48235953  0.50595188]
  [ 0.45700786  0.48172203  0.50987113]
  [ 0.39034119  0.48172203  0.50987113]]

 ..., 
 [[ 0.54723924  0.50965768  0.46340412]
  [ 0.54954123  0.50801635  0.46210581]
  [ 0.54977649  0.50505251  0.45754957]
  ..., 
  [ 0.14575295  0.47716442  0.515113  ]
  [ 0.14922746  0.47748318  0.51315337]
  [ 0.14922746  0.47748318  0.51315

[[[ 0.19145492  0.49211225  0.49154884]
  [ 0.19537647  0.49211228  0.49154884]
  [ 0.21078433  0.48951471  0.49612939]
  ..., 
  [ 0.14245491  0.48231199  0.49485543]
  [ 0.21159217  0.47775438  0.50009733]
  [ 0.25440785  0.46959543  0.50470221]]

 [[ 0.21381177  0.49015221  0.49221015]
  [ 0.20782353  0.48882967  0.48895225]
  [ 0.20249413  0.48983344  0.49416977]
  ..., 
  [ 0.14245491  0.48231199  0.49485543]
  [ 0.1993804   0.47807315  0.49813768]
  [ 0.25165883  0.4715555   0.5040409 ]]

 [[ 0.19529021  0.48098946  0.4915975 ]
  [ 0.20705491  0.48098946  0.4915975 ]
  [ 0.21069805  0.47839189  0.49617806]
  ..., 
  [ 0.15034118  0.48787341  0.49483112]
  [ 0.19258431  0.48851094  0.49091187]
  [ 0.25225884  0.48231199  0.49485543]]

 ..., 
 [[ 0.12075687  0.4865984   0.50266957]
  [ 0.11768628  0.48039949  0.50661319]
  [ 0.10664706  0.48267829  0.50399226]
  ..., 
  [ 0.11598431  0.49279732  0.49872601]
  [ 0.11481177  0.49083725  0.49938732]
  [ 0.1085451   0.48691714  0.50070

[[[ 0.42963532  0.44252101  0.52309746]
  [ 0.47469416  0.4411509   0.50874311]
  [ 0.71160787  0.44279224  0.51004142]
  ..., 
  [ 0.21052158  0.47292557  0.51839525]
  [ 0.20129412  0.46832043  0.5125407 ]
  [ 0.23675688  0.46540412  0.51908088]]

 [[ 0.40610591  0.44252101  0.52309746]
  [ 0.42880785  0.443111    0.50808179]
  [ 0.55559611  0.43659329  0.51398498]
  ..., 
  [ 0.20651765  0.48696467  0.51180637]
  [ 0.20417255  0.48304459  0.51312906]
  [ 0.24193726  0.47848696  0.51837093]]

 [[ 0.37910196  0.44220227  0.52505708]
  [ 0.41979218  0.44115093  0.50874311]
  [ 0.39990589  0.43855339  0.51332366]
  ..., 
  [ 0.21872942  0.48664594  0.51376599]
  [ 0.21985884  0.48304456  0.51312906]
  [ 0.25992551  0.47684565  0.51707262]]

 ..., 
 [[ 0.46958038  0.46996173  0.51383901]
  [ 0.47156081  0.46016145  0.51714563]
  [ 0.37518433  0.46736416  0.51841956]
  ..., 
  [ 0.39406669  0.46508536  0.5210405 ]
  [ 0.40231374  0.45920521  0.52302444]
  [ 0.42543924  0.46508536  0.52104

KeyboardInterrupt: 

### Extract HOG Features:

In [36]:
orient = 32
pix_per_cell = 16
cell_per_block = 2
hog_channel = "ALL"  # Can be 0, 1, 2, or "ALL"

t=time.time()
car_hog_features = extract_hog_features(vehicle_list, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
notcar_hog_features = extract_hog_features(non_vehicle_list, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)

In [37]:
print(len(car_color_features))
print(len(car_hog_features))
print(len(notcar_color_features))
print(len(notcar_hog_features))

8792
8792
5068
5068


In [38]:
# Warning! Pay Attention!
# Note that car_all_features is a 'list'. Each element of this list represents the combined
# features of a single image. 

# Combine color and hog features for images labeled as CAR
car_all_features=[]
for colorf,hogf in zip(car_color_features,car_hog_features):
    car_all_features.append(np.concatenate((colorf, hogf)))

In [39]:
# Combine color and hog features for images labeled as NONCAR
notcar_all_features=[]
for colorf, hogf in zip(notcar_color_features, notcar_hog_features):
    notcar_all_features.append(np.concatenate((colorf,hogf)))

In [40]:
#print(notcar_all_features[0])
#print(car_all_features[0])

In [41]:
print(max(car_color_features[0]))

2273.0


In [42]:
# Stack the CAR and NONCAR
X = np.vstack((car_all_features, notcar_all_features)).astype(np.float64)                        

In [43]:
# Stack the corresponding labels:
y = np.hstack((np.ones(len(car_all_features)), np.zeros(len(notcar_all_features))))

In [44]:
# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rand_state)

In [45]:
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)

# Apply the scaler to X
scaled_X = X_scaler.transform(X)    

In [46]:
# Fit a per-column scaler only on the training data
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X_train and X_test
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [47]:
X_train[0]

array([ 0.8597744 ,  1.90931008, -1.77953398, ..., -0.62718963,
       -0.40454714, -0.62958318])

In [48]:
print('Using spatial binning of:',spatial, 'and', histbin,'histogram bins')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Train Accuracy of SVC = ', round(svc.score(X_train, y_train), 4))
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample


Using spatial binning of: 32 and 32 histogram bins
Feature vector length: 6624
12.58 Seconds to train SVC...
Train Accuracy of SVC =  1.0
Test Accuracy of SVC =  0.9996


In [49]:
t=time.time()
n_predict = 100
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these' 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

My SVC predicts:  [ 0.  1.  1.  1.  1.  1.  1.  1.  0.  1.  0.  0.  1.  1.  0.  0.  1.  1.
  0.  1.  0.  1.  1.  0.  1.  0.  0.  1.  0.  1.  1.  1.  1.  1.  1.  0.
  0.  1.  1.  1.  0.  1.  0.  1.  1.  0.  1.  1.  1.  1.  0.  1.  1.  1.
  1.  1.  0.  1.  0.  1.  0.  0.  1.  0.  0.  1.  1.  1.  1.  0.  1.  0.
  1.  0.  0.  1.  1.  1.  1.  1.  0.  1.  0.  0.  1.  0.  1.  0.  1.  1.
  1.  1.  0.  0.  1.  1.  0.  1.  1.  1.]
For theselabels:  [ 0.  1.  1.  1.  1.  1.  1.  1.  0.  1.  0.  0.  1.  1.  0.  0.  1.  1.
  0.  1.  0.  1.  1.  0.  1.  0.  0.  1.  0.  1.  1.  1.  1.  1.  1.  0.
  0.  1.  1.  1.  0.  1.  0.  1.  1.  0.  1.  1.  1.  1.  0.  1.  1.  1.
  1.  1.  0.  1.  0.  1.  0.  0.  1.  0.  0.  1.  1.  1.  1.  0.  1.  0.
  1.  0.  0.  1.  1.  1.  1.  1.  0.  1.  0.  0.  1.  0.  1.  0.  1.  1.
  1.  1.  0.  0.  1.  1.  0.  1.  1.  1.]
0.0137 Seconds to predict 100 labels with SVC


In [50]:
# Save model to a file
import pickle

# Save the model:
with open('model_YCrCb_hist_range.pickle', 'wb') as handle:
    pickle.dump(svc, handle)
    
# Save the scalar function used to normalize the data:
with open('scalar_YCrCb_hist_range.pickle', 'wb') as handle:
    pickle.dump(X_scaler, handle)
    

In [ ]:
# Load a model from a pickle file:
with open('model_YCrCb.pickle', 'rb') as handle:
    inmodel = pickle.load(handle)

In [ ]:
print('Test Accuracy of loaded model = ', round(inmodel.score(X_test, y_test), 4))

In [ ]:
# Some References:
# 1. Pickle file write/read:
# https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
# 2. 